In [ ]:
#| hide
from onprem.core import *

# OnPrem

> A tool for running on-premise large language models on non-public data

**OnPrem** is a Python package that makes it easier to run large language models (LLMs) on non-public or sensitive data and in air-gapped networks.

## Install

```sh
pip install onprem
```

## How to use

**Coming Soon**